In [1]:
# %pip install openai==0.28
# %pip install "weaviate-client==3.*"
# %pip install -U langchain-openai

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import openai
import weaviate
env_vars = {
    "OPENAI_API_KEY" : "",
    "WEAVIATE_API_KEY" : ""
}
openai.api_key = env_vars['OPENAI_API_KEY']
auth_config = weaviate.auth.AuthApiKey(
    api_key= env_vars['WEAVIATE_API_KEY']
)

In [4]:
weaviate_client = weaviate.Client(
    url = "https://tashu-ikyhn3en.weaviate.network",
    auth_client_secret=auth_config,
    additional_headers={"X-OpenAI-Api-Key": env_vars['OPENAI_API_KEY']}
)

c:\Users\Hlo\venv\lib\site-packages\weaviate\warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.4.4.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [16]:
prompt="Hello, how can I help you today?"
messages = [{"role": "user", "content": prompt}]
def openai_chat_completion(messages):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0, 
    )
    response = completion.choices[0].message['content']
    return response

In [6]:
import json 
def load_json_file(filename):
    with open(filename) as f:
        file = json.load(f)
    return file
filename = 'intents.json'
json_file = load_json_file(filename)

In [7]:
import pandas as pd 
tag_list = []
pattern_list = []
response_list = []
intents = json_file['intents']
for intent in intents:
    tag = intent["tag"]
    patterns = intent["patterns"]
    responses = intent["responses"]
    for pattern in patterns:
        tag_list.append(tag)
        pattern_list.append(pattern)
        response_list.append(', '.join(responses))
dic = {
    "Tag": tag_list,
    "Pattern": pattern_list,
    "Response": response_list
}
df = pd.DataFrame(dic)
df.to_csv("data.csv",index=False)
df

,Tag,Pattern,Response
0,greeting,Hi,"Hello!, Good to see you again!, Hi there, how ..."
1,greeting,How are you?,"Hello!, Good to see you again!, Hi there, how ..."
2,greeting,Is anyone there?,"Hello!, Good to see you again!, Hi there, how ..."
3,greeting,Hello,"Hello!, Good to see you again!, Hi there, how ..."
4,greeting,Good day,"Hello!, Good to see you again!, Hi there, how ..."
...,...,...,...
400,ragging,ragging history,We are Proud to tell you that our college prov...
401,ragging,ragging incidents,We are Proud to tell you that our college prov...
402,hod,hod,"HODs differ for each branch, please be more sp..."
403,hod,hod name,"HODs differ for each branch, please be more sp..."


In [8]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader('data.csv')
data = loader.load()

In [9]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key = env_vars['OPENAI_API_KEY'])

In [10]:
from langchain.vectorstores import Weaviate
vectorstore = Weaviate.from_documents(data, embeddings, client = weaviate_client, by_text= True)

c:\Users\Hlo\venv\lib\site-packages\pydantic\main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [11]:
from langchain_openai import OpenAI
llm = OpenAI(openai_api_key= env_vars['OPENAI_API_KEY'],temperature=0)

In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [13]:
context = """
    Use the dataset {data} and generate as many as possible questions with answers. Answers about application deadlines, required documents, tuition fees, scholarship opportunities. Don't try to make up question answers on your own that are not in the datset. keep the answer as concise as possible.
"""
template = """Use the following pieces of context to answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use fifty words maximum. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type='stuff',
    retriever=vectorstore.as_retriever(search_type = "mmr"),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [19]:
from bert_score import score
def generate(question):
    result = qa_chain.invoke({"query": question})
    P, R, F1 = score([result['result']], [question], lang="en", verbose=False)
    return result['result'], f"Precision: {P.mean()}\nRecall: {R.mean()}\nF1: {F1.mean()}\n"

In [20]:
import gradio as gr
demo = gr.Interface(fn=generate, 
                    inputs=[gr.Textbox(label="You")],
                    outputs=[gr.Textbox(label="Bot")],
                    title="FAQ Chatbot",
                    description="Answers about application deadlines, required documents, tuition fees, scholarship opportunities, etc. Ask your questions and get instant answers!"
)

gr.close_all()
demo.launch()

c:\Users\Hlo\venv\lib\site-packages\gradio\routes.py:816: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
c:\Users\Hlo\venv\lib\site-packages\fastapi\applications.py:4495: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  return self.router.on_event(event_type)


Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\Hlo\venv\lib\site-packages\starlette\templating.py:172: DeprecationWarning: The `name` is not the first parameter anymore. The first parameter should be the `Request` instance.
Replace `TemplateResponse(name, {"request": request})` by `TemplateResponse(request, name)`.
  warnings.warn(
Exception in thread Thread-17 (_do_normal_analytics_request):
Traceback (most recent call last):
  File "c:\Users\Hlo\venv\lib\site-packages\httpcore\_exceptions.py", line 10, in map_exceptions
    yield
  File "c:\Users\Hlo\venv\lib\site-packages\httpcore\_backends\sync.py", line 206, in connect_tcp
    sock = socket.create_connection(
  File "C:\Users\Hlo\AppData\Local\Programs\Python\Python310\lib\socket.py", line 845, in create_connection
    raise err
  File "C:\Users\Hlo\AppData\Local\Programs\Python\Python310\lib\socket.py", line 833, in create_connection
    sock.connect(sa)
TimeoutError: timed out

The above exception was the direct cause of the following exception:

Traceback (most rec